In [48]:
import numpy as np
import copy

class CPU():
    def __init__(self):
        pass
    def genTasks(self, task_cnt=5, idle_time=5, max_time=10):
        self.task_cnt = task_cnt
        delta_time = np.random.randint(idle_time, size=(task_cnt))
        acc_time = np.cumsum(delta_time)
        cpu_time = np.random.randint(max_time-1, size=(task_cnt))+1
        self.tasks = [{'come_time': acc_time[i], 
                       'cpu_time': cpu_time[i],
                       'status': 'inactive',
                       'start_serving_time': -1,
                       'running_time': 0,
                       'end_time': -1,} for i in range(task_cnt)]
        self.tasks_ = copy.deepcopy(self.tasks)
    def reset(self):
        self.tasks = copy.deepcopy(self.tasks_)
        self.time = 1
    
    def is_finished(self):
        return len([i for i in self.tasks if i['status'] == 'done']) == len(self.tasks)
    
    def get_active_tasks(self):
        return [idx for idx,i in enumerate(self.tasks) if i['status'] == 'active']
    
    def activate_first_come_task(self):
        idx = -1
        min_cometime = 99999
        for idx_, task in enumerate(self.tasks):
            if task['status'] == 'inactive' and task['come_time'] < min_cometime and task['come_time'] <= self.time:
                idx = idx_
                min_cometime = task['come_time']
        if idx == -1:
            return False
    
        self.tasks[idx]['status'] = 'active'
        self.tasks[idx]['start_serving_time'] = self.time
        return True
    
    def activate_all_come_task(self):
        for idx, task in enumerate(self.tasks):
            if task['status'] == 'inactive' and task['come_time'] <= self.time:
                self.tasks[idx]['status'] = 'active'
                self.tasks[idx]['start_serving_time'] = self.time
    
    def activate_SJF_task(self):
        idx = -1
        min_cometime = 99999
        for idx_, task in enumerate(self.tasks):
            if task['status'] == 'inactive' and task['running_time'] < min_cometime and task['come_time'] <= self.time:
                idx = idx_
                min_cometime = task['running_time']
        if idx == -1:
            return False
    
        self.tasks[idx]['status'] = 'active'
        self.tasks[idx]['start_serving_time'] = self.time
        return True
    
    def activate_HRN_task(self):
        idx = -1
        min_cometime = -1
        for idx_, task in enumerate(self.tasks):
            t = (self.time - task['come_time']) / task['running_time']
            if task['status'] == 'inactive' and t > min_cometime and task['come_time'] <= self.time:
                idx = idx_
                min_cometime = t
        if idx == -1:
            return False
    
        self.tasks[idx]['status'] = 'active'
        self.tasks[idx]['start_serving_time'] = self.time
        return True
    
    def FCFS_step(self):
        if self.is_finished():
            return
        
        activated_tasks = self.get_active_tasks()
        assert len(activated_tasks) <= 1 # FCFS's condition
        
        if len(activated_tasks) == 1:
            # get ongoing task
            activated_task = activated_tasks[0]
            self.tasks[activated_task]['running_time'] += 1
            
            # check if done
            if self.tasks[activated_task]['running_time'] >= self.tasks[activated_task]['cpu_time']:
                self.tasks[activated_task]['status'] = 'done'
                self.tasks[activated_task]['end_time'] = self.time
                self.activate_first_come_task()
        else:
            self.activate_first_come_task()
            
        self.time += 1
        return
    
    def RR_step(self):
        if self.is_finished():
            return
        
        self.activate_all_come_task()
        activated_tasks = self.get_active_tasks()
        
        if len(activated_tasks) >= 1:
            # get ongoing task
            for activated_task in activated_tasks:
                # CPU sharing
                self.tasks[activated_task]['running_time'] += 1.0 / len(activated_tasks)

                # check if done
                if self.tasks[activated_task]['running_time'] >= self.tasks[activated_task]['cpu_time']:
                    self.tasks[activated_task]['status'] = 'done'
                    self.tasks[activated_task]['end_time'] = self.time
            
        self.time += 1
        return
    
    def SJF_step(self):
        if self.is_finished():
            return
        
        activated_tasks = self.get_active_tasks()
        assert len(activated_tasks) <= 1 # SJF's condition
        
        if len(activated_tasks) == 1:
            # get ongoing task
            activated_task = activated_tasks[0]
            self.tasks[activated_task]['running_time'] += 1
            
            # check if done
            if self.tasks[activated_task]['running_time'] >= self.tasks[activated_task]['cpu_time']:
                self.tasks[activated_task]['status'] = 'done'
                self.tasks[activated_task]['end_time'] = self.time
                self.activate_SJF_task()
        else:
            self.activate_SJF_task()
            
        self.time += 1
        return
    
    def HRN_step(self):
        if self.is_finished():
            return
        
        activated_tasks = self.get_active_tasks()
        assert len(activated_tasks) <= 1 # HRN's condition
        
        if len(activated_tasks) == 1:
            # get ongoing task
            activated_task = activated_tasks[0]
            self.tasks[activated_task]['running_time'] += 1
            
            # check if done
            if self.tasks[activated_task]['running_time'] >= self.tasks[activated_task]['cpu_time']:
                self.tasks[activated_task]['status'] = 'done'
                self.tasks[activated_task]['end_time'] = self.time
                self.activate_HRN_task()
        else:
            self.activate_HRN_task()
            
        self.time += 1
        return
    
    def gen_timesheet(self, method='FCFS'):
        self.timesheet = []
        import copy
        for _ in range(45):
            self.timesheet.append(copy.deepcopy(self.tasks))
            if method == 'FCFS':
                cpu.FCFS_step()
            elif method == 'RR':
                cpu.RR_step()
            elif method == 'SJF':
                cpu.SJF_step()
            elif method == 'HRN':
                cpu.HRN_step()
                
    def get_comingtime(self):
        return ', '.join([str(i['come_time']) for i in self.timesheet[-1]])
    def get_startservingtime(self):
        return ', '.join([str(i['start_serving_time']) for i in self.timesheet[-1]])
    def get_endtime(self):
        return ', '.join([str(i['end_time']) for i in self.timesheet[-1]])
    def get_turnroundtime(self):
        a = [(i['end_time']) for i in self.timesheet[-1]]
        b = [(i['come_time']) for i in self.timesheet[-1]]
        c = np.array(a) - np.array(b)
        return ', '.join([str(i) for i in c])
    def get_weightedturnroundtime(self):
        a = [(i['end_time']) for i in self.timesheet[-1]]
        b = [(i['come_time']) for i in self.timesheet[-1]]
        c = np.array(a) - np.array(b)
        d = [(i['start_serving_time']) for i in self.timesheet[-1]]
        e = np.array(a) - np.array(d)
        return ', '.join([str(round(i/j,1)) for i,j in zip(c, e)])
        
cpu = CPU()
cpu.genTasks()
cpu.reset()
cpu.gen_timesheet()

In [ ]:
import bimpy
import numpy as np

ctx = bimpy.Context()
ctx.init(800, 470, "EXP02")
with ctx:
        bimpy.themes.set_light_theme()
        
TASK_CNT = bimpy.Int(6)
IDLE_TIME = bimpy.Int(5)
MAX_RUNTIME = bimpy.Int(10)
cpu = CPU()
cpu.genTasks(task_cnt=TASK_CNT.value, 
             idle_time=IDLE_TIME.value, 
             max_time=MAX_RUNTIME.value)
cpu.reset()
cpu.gen_timesheet()

status2color = {
    'inactive': 0xaaaaaa, # gray
    'active': 0x4bb43c, # green
    'done': 0x555555, # black
    'notcoming': 0x000000,
}

while not ctx.should_close():
        ctx.new_frame()

        bimpy.set_next_window_pos(bimpy.Vec2(20, 20), bimpy.Condition.Once)
        bimpy.set_next_window_size(bimpy.Vec2(750, 200), bimpy.Condition.Once)
        bimpy.begin("CPU")
        bimpy.text("CPU Running Tasks (Tasks are sorted by comming time)")
        bimpy.text("Gray is waiting, green is running, dark gray is done.")
        
        window_pos = bimpy.get_window_pos() + bimpy.Vec2(30, 70)
        
        for idx1 in range(cpu.task_cnt):
            for idx2 in range(45):
                point = bimpy.Vec2(idx2*15, idx1*15)
                if idx2 < cpu.timesheet[0][idx1]['come_time']:
                    color = status2color['notcoming']
                else:
                    color = status2color[cpu.timesheet[idx2][idx1]['status']]
                bimpy.add_circle_filled(window_pos + point, 5, 0xAF000000 + color, 100)

                
        bimpy.end()

        bimpy.set_next_window_pos(bimpy.Vec2(20, 230), bimpy.Condition.Once)
        bimpy.set_next_window_size(bimpy.Vec2(750, 240), bimpy.Condition.Once)
        bimpy.begin("Controls")
        
        bimpy.text("Experiment2 Chengxuan Ying 201785071")
        bimpy.input_int("Cpu Task Count", TASK_CNT)
        bimpy.input_int("Task Coming Interval", IDLE_TIME)
        bimpy.input_int("Max Generated Task Time", MAX_RUNTIME)
        if bimpy.button("Generate data and use FCFS"):
            cpu.genTasks(task_cnt=TASK_CNT.value, 
                         idle_time=IDLE_TIME.value, 
                         max_time=MAX_RUNTIME.value)
            cpu.reset()
            cpu.gen_timesheet()
        
        bimpy.same_line()
        if bimpy.button("use FCFS"):
            cpu.reset()
            cpu.gen_timesheet()
            
        bimpy.same_line()
        if bimpy.button("use RR"):
            cpu.reset()
            cpu.gen_timesheet(method='RR')
            
        bimpy.same_line()
        if bimpy.button("use SJF"):
            cpu.reset()
            cpu.gen_timesheet(method='SJF')
            
        bimpy.same_line()
        if bimpy.button("use HRN"):
            cpu.reset()
            cpu.gen_timesheet(method='HRN')
        
        bimpy.text('coming time: ' + cpu.get_comingtime())
        bimpy.text('start serving time: ' + cpu.get_startservingtime())
        bimpy.text('ending time: ' + cpu.get_endtime())
        bimpy.text('turn round time: ' + cpu.get_turnroundtime())
        bimpy.text('weighted turn round time: ' + cpu.get_weightedturnroundtime())
        bimpy.end()
        ctx.render()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:67: RuntimeWarning: divide by zero encountered in long_scalars
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in long_scalars
